In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf  # Library to fetch data from Yahoo Finance

# Download the WTI futures time series (CL=F symbol on Yahoo Finance)
# Set your desired date range
data = yf.download("CL=F", start="2023-01-01", end="2023-12-31", interval="1m")  # 1-minute interval for intraday data

# Ensure the downloaded data has 'Timestamp' and 'Price' columns by renaming appropriately
data = data.rename(columns={"Adj Close": "Price"})  # Use 'Adj Close' as the price
data.reset_index(inplace=True)  # Reset index to make 'Datetime' a column
data = data[['Datetime', 'Price']]  # Keep only necessary columns

# Sort the data by date and time if needed
data = data.sort_values('Datetime').reset_index(drop=True)

# Calculate intraday volatility (standard deviation of returns) over 30-minute intervals
data.set_index('Datetime', inplace=True)
data['Return'] = data['Price'].pct_change()  # Calculate returns

# Resample the time series into 30-minute intervals
volatility_30min = data['Return'].rolling('30T').std() * np.sqrt(30)  # Annualize if needed

# Detect anomalies - simple threshold based on a high standard deviation (e.g., 3 sigma)
threshold = volatility_30min.mean() + 3 * volatility_30min.std()  # Adjustable based on tolerance
anomalies = volatility_30min[volatility_30min > threshold]

# Visualize volatility and anomalies
plt.figure(figsize=(12, 6))
plt.plot(volatility_30min, label="30-Min Volatility")
plt.scatter(anomalies.index, anomalies, color='red', label="Volatility Spikes (Anomalies)", marker='o')
plt.xlabel("Time")
plt.ylabel("Volatility")
plt.title("30-Min Volatility with Identified Spikes")
plt.legend()
plt.show()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['CL=F']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1m 2023-01-01 -> 2023-12-31) (Yahoo error = "1m data not available for startTime=1672549200 and endTime=1703998800. Only 8 days worth of 1m granularity data are allowed to be fetched per request.")')


KeyError: "['Datetime'] not in index"